In [51]:
%matplotlib inline
import os
from pathlib import Path
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils import data
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler


In [10]:
p = Path('dataset/iris.data')
df = pd.read_csv(p)
feature_cols = ['sepal_length', 'sepal_width','petal_length','petal_witdh']
target_cols = ['class']

df.loc[df['class']=='Iris-setosa', 'class'] = 0
df.loc[df['class']=='Iris-versicolor', 'class'] = 1
df.loc[df['class']=='Iris-virginica', 'class'] = 2
# print(df)

In [77]:
class IrisDataset(data.Dataset):
    def __init__(
            self, path:str, feature_cols:list, 
            target_cols:list, clazz:list, 
            transforms_feature=None, transforms_target=None):
        
        self.path = Path(path)
        self.dframe = pd.read_csv(self.path)
        self.feature_cols = feature_cols
        self.target_cols = target_cols
        self.clazz = clazz
        self.transforms_feature = transforms_feature
        self.transforms_target = transforms_target
        
        self.__normalize_target()
        self.class_to_idx = self.__class_to_label()
        self.idx_to_class = self.__idx_to_class()
    
    def __len__(self):
        return len(self.dframe)
    
    def __class_to_label(self):
        mapz = [(val, idx) for idx, val in enumerate(self.clazz)]
        return dict(mapz)
    
    def __idx_to_class(self):
        mapz = [(idx, val) for idx, val in enumerate(self.clazz)]
        return dict(mapz)
    
    def __normalize_target(self):
        cat_type = CategoricalDtype(categories=self.clazz, ordered=True)
        self.dframe[self.target_cols[0]] = self.dframe[self.target_cols[0]].astype(cat_type).cat.codes
    
    def __getitem__(self, idx):
        feature = self.dframe[self.feature_cols].iloc[idx].values
        target = self.dframe[self.target_cols].iloc[idx].values
        target = np.squeeze(target)
        
        if self.transforms_feature:
            feature = self.transforms_feature(feature)
        if self.transforms_target:
            target = self.transforms_target(target)
            
        return feature, target


def indice_splitter(dataset, valid_size, shuflle=True):
    num_data = len(dataset)
    indices = list(range(num_data))
    split = int(np.floor(valid_size * num_data))
    if shuflle:
        np.random.seed(1)
        np.random.shuffle(indices)
    train_idx, valid_idx = indices[split:], indices[:split]
    return train_idx, valid_idx

class NumpyToFloatTensor(object):
    def __call__(self, param):
        return torch.from_numpy(param.astype(np.float32)).float()
    
class NumpyToLongTensor(object):
    def __call__(self, param):
        return torch.from_numpy(param.astype(np.long)).long()
        
        
path = 'dataset/iris.data'
feature_cols = ['sepal_length', 'sepal_width','petal_length','petal_witdh']
target_cols = ['class']
clazz = ["Iris-setosa", "Iris-versicolor", "Iris-virginica"]

iris_dataset = IrisDataset(
    path, feature_cols, 
    target_cols, clazz, 
    transforms_feature=NumpyToFloatTensor(), transforms_target=NumpyToLongTensor())

train_idx, valid_idx = indice_splitter(iris_dataset, valid_size=0.2)

train_loader = data.DataLoader(iris_dataset, batch_size=32, sampler=SubsetRandomSampler(train_idx), num_workers=0)
valid_loader = data.DataLoader(iris_dataset, batch_size=32, sampler=SubsetRandomSampler(valid_idx), num_workers=0)

In [54]:
# loss = nn.CrossEntropyLoss()
# input = torch.randn(3, 5, requires_grad=True)
# target = torch.empty(3, dtype=torch.long).random_(5)
# # output = loss(input, target)
data_iter = iter(train_loader)
x, y = next(data_iter)
# torch.utils.data.random_split(iris_dataset, len(iris_dataset))

tensor([[6.0000, 3.0000, 4.8000, 1.8000],
        [7.7000, 3.0000, 6.1000, 2.3000],
        [5.5000, 2.4000, 3.8000, 1.1000],
        [5.7000, 2.8000, 4.5000, 1.3000],
        [5.0000, 2.3000, 3.3000, 1.0000],
        [5.5000, 2.5000, 4.0000, 1.3000],
        [4.8000, 3.1000, 1.6000, 0.2000],
        [4.6000, 3.6000, 1.0000, 0.2000],
        [4.6000, 3.1000, 1.5000, 0.2000],
        [5.5000, 3.5000, 1.3000, 0.2000],
        [6.5000, 3.0000, 5.5000, 1.8000],
        [4.8000, 3.0000, 1.4000, 0.1000],
        [5.7000, 4.4000, 1.5000, 0.4000],
        [5.6000, 2.8000, 4.9000, 2.0000],
        [4.8000, 3.4000, 1.9000, 0.2000],
        [4.9000, 3.1000, 1.5000, 0.1000],
        [6.7000, 3.0000, 5.2000, 2.3000],
        [5.9000, 3.2000, 4.8000, 1.8000],
        [4.6000, 3.2000, 1.4000, 0.2000],
        [6.1000, 2.9000, 4.7000, 1.4000],
        [6.3000, 2.9000, 5.6000, 1.8000],
        [5.5000, 2.4000, 3.7000, 1.0000],
        [6.4000, 2.9000, 4.3000, 1.3000],
        [6.7000, 3.1000, 4.7000, 1

In [55]:


net = nn.Sequential(
    nn.Linear(4,100),
    nn.Linear(100,3)
)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)

for epoch in range(100):
    for idx, (x_train, y_train) in enumerate(train_loader):
        out = net(x_train)
        loss = criterion(out, y_train)
        loss.backward()
        optimizer.step()
        print(loss)

tensor(1.8174, grad_fn=<NllLossBackward>)
tensor(1.2106, grad_fn=<NllLossBackward>)
tensor(1.7663, grad_fn=<NllLossBackward>)
tensor(0.9119, grad_fn=<NllLossBackward>)
tensor(1.0570, grad_fn=<NllLossBackward>)
tensor(0.6807, grad_fn=<NllLossBackward>)
tensor(0.5482, grad_fn=<NllLossBackward>)


tensor(0.3834, grad_fn=<NllLossBackward>)
tensor(0.5320, grad_fn=<NllLossBackward>)
tensor(0.4112, grad_fn=<NllLossBackward>)
tensor(0.5653, grad_fn=<NllLossBackward>)
tensor(0.3236, grad_fn=<NllLossBackward>)


tensor(0.3790, grad_fn=<NllLossBackward>)
tensor(0.2627, grad_fn=<NllLossBackward>)
tensor(0.1878, grad_fn=<NllLossBackward>)
tensor(0.3983, grad_fn=<NllLossBackward>)
tensor(0.1453, grad_fn=<NllLossBackward>)
tensor(0.1881, grad_fn=<NllLossBackward>)


tensor(0.1375, grad_fn=<NllLossBackward>)
tensor(0.3825, grad_fn=<NllLossBackward>)
tensor(0.2569, grad_fn=<NllLossBackward>)
tensor(0.3517, grad_fn=<NllLossBackward>)
tensor(0.2267, grad_fn=<NllLossBackward>)
tensor(0.2212, grad_fn=<NllLossBackward>)
tensor(0.2562, grad_fn=<NllLossBackward>)


tensor(0.2273, grad_fn=<NllLossBackward>)
tensor(0.1197, grad_fn=<NllLossBackward>)
tensor(0.3072, grad_fn=<NllLossBackward>)
tensor(0.2332, grad_fn=<NllLossBackward>)
tensor(0.0285, grad_fn=<NllLossBackward>)
tensor(0.1922, grad_fn=<NllLossBackward>)
tensor(0.2591, grad_fn=<NllLossBackward>)


...

tensor(29.3111, grad_fn=<NllLossBackward>)
tensor(0., grad_fn=<NllLossBackward>)
tensor(6.9258, grad_fn=<NllLossBackward>)
tensor(7.3921, grad_fn=<NllLossBackward>)
tensor(19.5669, grad_fn=<NllLossBackward>)
tensor(0., grad_fn=<NllLossBackward>)
tensor(6.7516, grad_fn=<NllLossBackward>)
tensor(5.4999, grad_fn=<NllLossBackward>)


tensor(1.6739, grad_fn=<NllLossBackward>)
tensor(11.9256, grad_fn=<NllLossBackward>)
tensor(6.8600, grad_fn=<NllLossBackward>)
tensor(2.3973, grad_fn=<NllLossBackward>)
tensor(9.4613, grad_fn=<NllLossBackward>)
tensor(4.2806, grad_fn=<NllLossBackward>)
tensor(9.8029, grad_fn=<NllLossBackward>)
tensor(5.2732, grad_fn=<NllLossBackward>)


tensor(1.6882, grad_fn=<NllLossBackward>)
tensor(9.7136, grad_fn=<NllLossBackward>)
tensor(0.0004, grad_fn=<NllLossBackward>)
tensor(0., grad_fn=<NllLossBackward>)
tensor(0., grad_fn=<NllLossBackward>)
tensor(30.9763, grad_fn=<NllLossBackward>)
tensor(23.9165, grad_fn=<NllLossBackward>)
tensor(2.1981, grad_fn=<NllLossBackward>)


tensor(23.3613, grad_fn=<NllLossBackward>)
tensor(3.6892, grad_fn=<NllLossBackward>)
tensor(0.4491, grad_fn=<NllLossBackward>)
tensor(0., grad_fn=<NllLossBackward>)
tensor(18.0534, grad_fn=<NllLossBackward>)
tensor(0.0595, grad_fn=<NllLossBackward>)
tensor(1.6822, grad_fn=<NllLossBackward>)
tensor(11.0575, grad_fn=<NllLossBackward>)


tensor(11.9947, grad_fn=<NllLossBackward>)
tensor(13.4055, grad_fn=<NllLossBackward>)
tensor(21.4363, grad_fn=<NllLossBackward>)
tensor(5.6913, grad_fn=<NllLossBackward>)
tensor(8.5944, grad_fn=<NllLossBackward>)
tensor(0., grad_fn=<NllLossBackward>)


In [66]:
data_iter = iter(valid_loader)
x, y = next(data_iter)



In [94]:
out = net(x)
softmax = torch.nn.functional.softmax(out[1], dim=0)
output = torch.argmax(softmax)
label_network = iris_dataset.idx_to_class[output.item()]
label_network


'Iris-setosa'

In [95]:
label_truth = iris_dataset.idx_to_class[y[1].item()]
label_truth

'Iris-setosa'